<a href="https://colab.research.google.com/github/DaheePark0415/Econ512-Fall2024/blob/main/IO_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1(a).

In [29]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the data
df = pd.read_csv('product_data.csv')

# Check the structure of the data to ensure it has the correct columns
print(df.head())

   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [30]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from scipy.optimize import minimize


In [31]:
# Load the dataset
df = pd.read_csv('/content/product_data.csv')

# Show the first few rows of the dataset to verify
print(df.head())


   product_ID  nest     price     sugar  caffeine  market_share  \
0           1  Diet  2.814362  0.631224  6.752525      0.111141   
1           2  Diet  2.935735  0.004553  6.784396      0.081787   
2           3  Diet  2.467309  0.739947  5.761261      0.085727   
3           4  Diet  1.543958  0.103660  4.468299      0.007187   
4           5  Diet  1.495961  0.971926  4.052750      0.015912   

   caffeine_extract_price  corn_syrup_price  t  
0                0.267468          0.251714  1  
1                0.320000          0.253146  1  
2                0.252531          0.314781  1  
3                0.203220          0.227481  1  
4                0.156466          0.244453  1  


In [35]:
# Define the utility function with corrected handling for Diet and Regular using the 'nest' column
def utility(params, price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)  # Assuming the other category is Regular
    return alpha * price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Define the log-likelihood function for the multinomial logit model
def log_likelihood(params, df):
    # Calculate utility for each product in each time period
    df['utility'] = utility(params, df['price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by quantity (here using 'market_share')
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result = minimize(log_likelihood, initial_guess, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters:", result.x)


Estimated parameters: [-0.32866229  0.85382714  0.8080273  11.61252541  8.8465584 ]


# 1(b).

In [36]:
from sklearn.linear_model import LinearRegression

# Define the exogenous variables (instruments + other exogenous variables)
exog_vars = ['sugar', 'caffeine', 'caffeine_extract_price', 'corn_syrup_price']

# First stage: Regress price on instruments and other exogenous variables
model_stage_1 = LinearRegression()
model_stage_1.fit(df[exog_vars], df['price'])

# Get the predicted price from the first stage
df['predicted_price'] = model_stage_1.predict(df[exog_vars])


In [37]:
# Adjust the utility function to use the predicted price
def utility_iv(params, predicted_price, sugar, caffeine, nest):
    alpha, beta1, beta2, gamma_d, gamma_r = params
    diet = (nest == 'Diet').astype(int)
    regular = (nest != 'Diet').astype(int)
    return alpha * predicted_price + beta1 * sugar + beta2 * caffeine + gamma_d * diet + gamma_r * regular

# Adjust the log-likelihood function to use the predicted price
def log_likelihood_iv(params, df):
    # Calculate utility for each product in each time period using predicted price
    df['utility'] = utility_iv(params, df['predicted_price'], df['sugar'], df['caffeine'], df['nest'])

    # Exponentiate utility
    df['exp_utility'] = np.exp(df['utility'])

    # Group by time period, sum over products to get the denominator for each period
    df['sum_exp_util'] = df.groupby('t')['exp_utility'].transform('sum') + 1  # Add 1 for the outside option

    # Probability of choosing product j at time t
    df['prob'] = df['exp_utility'] / df['sum_exp_util']

    # Log of the probability weighted by market share
    df['log_prob'] = np.log(df['prob']) * df['market_share']

    # Negative log-likelihood (since we minimize)
    return -df['log_prob'].sum()

# Initialize the parameters: [alpha, beta1, beta2, gamma_d, gamma_r]
initial_guess_iv = [0.1, 0.1, 0.1, 0.1, 0.1]

# Estimate the parameters by minimizing the negative log-likelihood
result_iv = minimize(log_likelihood_iv, initial_guess_iv, args=(df,), method='BFGS')

# Print the estimated parameters
print("Estimated parameters (IV):", result_iv.x)


Estimated parameters (IV): [-1.33114723  1.03226555  1.02240006 14.35370063 12.09851264]


Problem 1(b) involves addressing endogeneity in the price variable. In the given context, the price of soft drinks is correlated with unobserved quality (𝜉𝑗𝑡​). This means that standard OLS estimators would be biased. To solve this issue, we can use instrumental variables (IV) for the price.

In this problem, the suggested instruments are:

Caffeine Extract Price and Corn Syrup Price.

These instruments are used because they affect the price but are not directly related to the unobserved demand shock (𝜉𝑗𝑡).

Approach:
Two-Stage Least Squares (2SLS): First, we will regress the endogenous variable (price) on the instruments to get the predicted prices. Then, we will use these predicted prices in the utility function to estimate the parameters.

Conditions for Valid Instruments:

Relevance: The instruments must be correlated with the endogenous variable (price).
Exogeneity: The instruments must not be correlated with the error term (unobserved quality 𝜉𝑗𝑡).

# 새 섹션